In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import optuna
import plotly.express as px
import random

# Load and prepare data
DATA_PATH = 'german_credit_data_cleaned.csv'
df = pd.read_csv(DATA_PATH)
y = df['Risk_good'].values
X_full = df.drop(columns=['Risk_good'])
sensitive = df['Sex_male'].values



In [2]:
GLOBAL_SEED = 39
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

In [3]:
# Define the custom objective function factory
def make_custom_obj(sens_array, alpha=0.5):
    def custom_obj(preds, dmatrix):
        labels = dmatrix.get_label()
        preds_prob = 1.0 / (1.0 + np.exp(-preds))
        grad_log = preds_prob - labels
        hess_log = preds_prob * (1.0 - preds_prob)
        mask_p = (sens_array == 1)
        mask_u = (sens_array == 0)
        n_p, n_u = mask_p.sum(), mask_u.sum()
        mean_p = preds_prob[mask_p].mean() if n_p > 0 else 0
        mean_u = preds_prob[mask_u].mean() if n_u > 0 else 0
        diff = mean_p - mean_u
        sign = np.sign(diff)
        sigmoid_deriv = preds_prob * (1.0 - preds_prob)
        grad_fair = np.zeros_like(preds)
        grad_fair[mask_p] = sign * (1.0 / n_p) * sigmoid_deriv[mask_p] if n_p > 0 else 0
        grad_fair[mask_u] = -sign * (1.0 / n_u) * sigmoid_deriv[mask_u] if n_u > 0 else 0
        hess_fair = np.zeros_like(preds)
        grad = alpha * grad_log + (1 - alpha) * grad_fair
        hess = alpha * hess_log + (1 - alpha) * hess_fair
        return grad, hess
    return custom_obj

# Define the objective function for Optuna

def create_objective_function(alpha,seed=GLOBAL_SEED):
    def objective(trial):
        # Suggest hyperparameters
        # alpha = trial.suggest_float('alpha', 0.0, 1.0)
        max_depth = trial.suggest_int('max_depth', 3, 7)
        eta = trial.suggest_float('eta', 0.01, 0.3)
        subsample = trial.suggest_float('subsample', 0.6, 1.0)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.6, 1.0)

        # Initialize cross-validation
        kf = KFold(n_splits=3, shuffle=True, random_state=seed)
        acc_scores = []
        dpd_scores = []

        # Perform 3-fold cross-validation
        for train_idx, val_idx in kf.split(X_full):
            # Split data
            X_train = X_full.iloc[train_idx].drop(columns=['Sex_male']).values
            y_train = y[train_idx]
            sens_train = sensitive[train_idx]
            X_val = X_full.iloc[val_idx].drop(columns=['Sex_male']).values
            y_val = y[val_idx]
            sens_val = sensitive[val_idx]

            # Create custom objective
            train_obj = make_custom_obj(sens_train, alpha=alpha)

            # Set up DMatrix
            dtrain = xgb.DMatrix(X_train, label=y_train)
            dval = xgb.DMatrix(X_val, label=y_val)

            # Train the model
            params = {
                'max_depth': max_depth,
                'eta': eta,
                'subsample': subsample,
                'colsample_bytree': colsample_bytree,
                'verbosity': 0,
                'seed': seed + trial.number
            }
            bst = xgb.train(
                params,
                dtrain,
                num_boost_round=100,
                obj=train_obj,
                evals=[(dtrain, 'train'), (dval, 'val')],
                early_stopping_rounds=10,
                verbose_eval=False
            )

            # Predict on validation set
            pred_prob = bst.predict(dval)
            pred = (pred_prob > 0.5).astype(int)

            # Compute accuracy and DPD
            acc = accuracy_score(y_val, pred)
            dpd_val = abs(pred[sens_val == 1].mean() - pred[sens_val == 0].mean())
            acc_scores.append(acc)
            dpd_scores.append(dpd_val)

        # Compute mean metrics across folds
        mean_acc = np.mean(acc_scores)
        mean_dpd = np.mean(dpd_scores)

        # Store metrics for analysis
        trial.set_user_attr('accuracy', mean_acc)
        trial.set_user_attr('dpd', mean_dpd)
        trial.set_user_attr('params', {
            'alpha': alpha,
            'max_depth': max_depth,
            'eta': eta,
            'subsample': subsample,
            'colsample_bytree': colsample_bytree
        })

        return mean_acc, mean_dpd
    return objective


In [4]:
# Create and run Optuna study for multi-objective optimization

def test_loss(a,dpd,accuracy):
    return (1-a)*(1-dpd) + a*accuracy

def is_pareto_efficient(acc, dpd):
    n_points = len(acc)
    is_efficient = [True] * n_points
    for i in range(n_points):
        for j in range(n_points):
            if (acc[j] >= acc[i] and dpd[j] <= dpd[i]) and (acc[j] > acc[i] or dpd[j] < dpd[i]):
                is_efficient[i] = False
                break
    return is_efficient

def normalise(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)
#adaptive weights based on distance

results = []
count_alpha = 30
trials_per_alpha = 100
sampler = optuna.samplers.NSGAIISampler(seed=GLOBAL_SEED)
study = optuna.create_study(directions=['maximize', 'minimize'], sampler=sampler)
objective = create_objective_function(alpha=0, seed=GLOBAL_SEED)  
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=0")
max_val = 0
for trial in study.best_trials:
    val = test_loss(0,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 0,
    'params': best_trial.user_attrs['params']
    })
study = optuna.create_study(directions=['maximize', 'minimize'], sampler=sampler)
objective = create_objective_function(alpha=1, seed=GLOBAL_SEED)  
study.optimize(objective, n_trials=trials_per_alpha)
print(f"Completed study for alpha=1")
max_val = 0
for trial in study.best_trials:
    val = test_loss(1,trial.values[1],trial.values[0])
    if val > max_val:
        max_val = val
        best_trial = trial
results.append({
    'accuracy': best_trial.values[0],
    'dpd': best_trial.values[1],
    'alpha': 1,
    'params': best_trial.user_attrs['params']
})
print(results)

max_acc = max([res['accuracy'] for res in results])
max_dpd = max([res['dpd'] for res in results])
min_acc = min([res['accuracy'] for res in results])
min_dpd = min([res['dpd'] for res in results])




for j in range(count_alpha):
    print(j)
    
    results.sort(key=lambda x: x['alpha'])
    max_dist = 0
    print(f"max_acc: {max_acc}, max_dpd: {max_dpd}, min_acc: {min_acc}, min_dpd: {min_dpd}")
    for i in range(len(results)-1):
        acc_dist = (normalise(results[i]['accuracy'],min_acc,max_acc) - normalise(results[i+1]['accuracy'],min_acc,max_acc))**2
        dpd_dist = (normalise(results[i]['dpd'],min_dpd,max_dpd) - normalise(results[i+1]['dpd'],min_dpd,max_dpd))**2

        dist = acc_dist + dpd_dist
        if dist > max_dist:
            max_dist = dist
            print(f"max dist: {max_dist}, acc_dist: {acc_dist}, dpd_dist: {dpd_dist}")
            alpha = (results[i]['alpha']+results[i+1]['alpha'])/2
    study = optuna.create_study(directions=['maximize', 'minimize'], sampler=sampler)
    objective = create_objective_function(alpha=alpha, seed=GLOBAL_SEED)  
    study.optimize(objective, n_trials=trials_per_alpha, )
    # print(f"Completed study for alpha={alpha}")
    max_val = 0
    for trial in study.best_trials:
        val = test_loss(alpha,trial.values[1],trial.values[0])
        if val > max_val:
            max_val = val
            best_trial = trial
        
    results.append({
        'accuracy': best_trial.values[0],
        'dpd': best_trial.values[1],
        'alpha': alpha,
        'params': best_trial.user_attrs['params']
    })
    max_acc = max(max_acc, best_trial.values[0])
    max_dpd = max(max_dpd, best_trial.values[1])
    min_acc = min(min_acc, best_trial.values[0])
    min_dpd = min(min_dpd, best_trial.values[1])
acc = [res['accuracy'] for res in results]
dpd = [res['dpd'] for res in results]
is_efficient = is_pareto_efficient(acc, dpd)
# Filter results to only include Pareto-optimal solutions
results = [res for res, efficient in zip(results, is_efficient) if efficient]
    

# Print summary of Pareto-optimal results
print("Pareto-optimal solutions:")
for res in results:
    print(f"Params: {res['params']}, Accuracy: {res['accuracy']:.4f}, DPD: {res['dpd']:.4f}")

# Prepare data for plotting
plot_df = pd.DataFrame({
    'DPD': [res['dpd'] for res in results],
    'Accuracy': [res['accuracy'] for res in results],
    'alpha': [res['alpha'] for res in results]
})



[I 2025-05-18 02:33:02,690] A new study created in memory with name: no-name-93e6a1f0-5ee9-4b5c-a3f1-b8946c5548ac
[I 2025-05-18 02:33:04,150] Trial 0 finished with values: [0.49999540487634525, 0.0] and parameters: {'max_depth': 5, 'eta': 0.2413907161719555, 'subsample': 0.9281607519593525, 'colsample_bytree': 0.6488199467320804}.
[I 2025-05-18 02:33:04,226] Trial 1 finished with values: [0.49999540487634525, 0.0] and parameters: {'max_depth': 6, 'eta': 0.16239922917572927, 'subsample': 0.7855633623240273, 'colsample_bytree': 0.7885782977698064}.
[I 2025-05-18 02:33:04,305] Trial 2 finished with values: [0.49999540487634525, 0.0] and parameters: {'max_depth': 6, 'eta': 0.2784425253370724, 'subsample': 0.9262011663001768, 'colsample_bytree': 0.977776700953335}.
[I 2025-05-18 02:33:04,386] Trial 3 finished with values: [0.49999540487634525, 0.0] and parameters: {'max_depth': 7, 'eta': 0.1303009309417961, 'subsample': 0.9303247849212601, 'colsample_bytree': 0.9785040475357668}.
[I 2025-05

Completed study for alpha=0


[I 2025-05-18 02:33:13,222] Trial 1 finished with values: [0.6542889352485809, 0.043523845757100776] and parameters: {'max_depth': 3, 'eta': 0.24559348505780326, 'subsample': 0.7797527960996078, 'colsample_bytree': 0.6419304816616404}.
[I 2025-05-18 02:33:13,348] Trial 2 finished with values: [0.6985874589885214, 0.04452275568868014] and parameters: {'max_depth': 7, 'eta': 0.2116727242601461, 'subsample': 0.6470347821074509, 'colsample_bytree': 0.9836397410016358}.
[I 2025-05-18 02:33:13,475] Trial 3 finished with values: [0.6950017308299099, 0.07761699774476764] and parameters: {'max_depth': 7, 'eta': 0.10067102888094598, 'subsample': 0.9841996835732718, 'colsample_bytree': 0.7154828141769964}.
[I 2025-05-18 02:33:13,572] Trial 4 finished with values: [0.6621435332824807, 0.04648156340083878] and parameters: {'max_depth': 4, 'eta': 0.14374294073740695, 'subsample': 0.9588334986644553, 'colsample_bytree': 0.6693448852214688}.
[I 2025-05-18 02:33:13,664] Trial 5 finished with values: [0

Completed study for alpha=1
[{'accuracy': 0.49999540487634525, 'dpd': 0.0, 'alpha': 0, 'params': {'alpha': 0, 'max_depth': 5, 'eta': 0.2413907161719555, 'subsample': 0.9281607519593525, 'colsample_bytree': 0.6488199467320804}}, {'accuracy': 0.7235788048389716, 'dpd': 0.06026591322496896, 'alpha': 1, 'params': {'alpha': 1, 'max_depth': 7, 'eta': 0.2610577064009086, 'subsample': 0.6653233396702862, 'colsample_bytree': 0.9559002852440293}}]
0
max_acc: 0.7235788048389716, max_dpd: 0.06026591322496896, min_acc: 0.49999540487634525, min_dpd: 0.0
max dist: 2.0, acc_dist: 1.0, dpd_dist: 1.0


[I 2025-05-18 02:33:25,893] Trial 1 finished with values: [0.6685721112755143, 0.04252066509016883] and parameters: {'max_depth': 6, 'eta': 0.13044284878431922, 'subsample': 0.638973784615826, 'colsample_bytree': 0.9471209539261723}.
[I 2025-05-18 02:33:25,989] Trial 2 finished with values: [0.6607236400731544, 0.041890910051932316] and parameters: {'max_depth': 4, 'eta': 0.28055000605952973, 'subsample': 0.7084546832757947, 'colsample_bytree': 0.9289224224249274}.
[I 2025-05-18 02:33:26,119] Trial 3 finished with values: [0.6721425223552765, 0.07155157376750455] and parameters: {'max_depth': 6, 'eta': 0.06759395598676973, 'subsample': 0.7028561792658066, 'colsample_bytree': 0.7187130439121279}.
[I 2025-05-18 02:33:26,223] Trial 4 finished with values: [0.680003247220716, 0.05308225328663562] and parameters: {'max_depth': 5, 'eta': 0.26323038071344057, 'subsample': 0.7657174027548451, 'colsample_bytree': 0.821368234061726}.
[I 2025-05-18 02:33:26,319] Trial 5 finished with values: [0.6

...

28
max_acc: 0.7235788048389716, max_dpd: 0.06026591322496896, min_acc: 0.49999540487634525, min_dpd: 0.0
max dist: 0.01714452065862096, acc_dist: 0.0171321682480249, dpd_dist: 1.2352410596059355e-05
max dist: 0.020423787316457692, acc_dist: 0.010472676638297167, dpd_dist: 0.009951110678160523
max dist: 0.03931663507543255, acc_dist: 0.03923368459217079, dpd_dist: 8.2950483261762e-05
max dist: 0.062166473558842875, acc_dist: 0.0621664734170249, dpd_dist: 1.4181797732120154e-10
max dist: 0.09895131953907549, acc_dist: 0.09012335524868446, dpd_dist: 0.008827964290391021


[I 2025-05-18 02:43:56,861] Trial 1 finished with values: [0.6707165023144106, 0.03431186549613302] and parameters: {'max_depth': 5, 'eta': 0.13521900332015727, 'subsample': 0.8204113359456918, 'colsample_bytree': 0.9747739808153921}.
[I 2025-05-18 02:43:56,932] Trial 2 finished with values: [0.6707165023144106, 0.0375172652532569] and parameters: {'max_depth': 7, 'eta': 0.2406638678815159, 'subsample': 0.7251822503923149, 'colsample_bytree': 0.8284139148249802}.
[I 2025-05-18 02:43:57,014] Trial 3 finished with values: [0.6707165023144106, 0.03431186549613302] and parameters: {'max_depth': 5, 'eta': 0.22522147193163852, 'subsample': 0.821743679282432, 'colsample_bytree': 0.8165339705546248}.
[I 2025-05-18 02:43:57,081] Trial 4 finished with values: [0.6707165023144106, 0.03431186549613302] and parameters: {'max_depth': 3, 'eta': 0.14134955975025967, 'subsample': 0.6953244062445976, 'colsample_bytree': 0.7926763166820063}.
[I 2025-05-18 02:43:57,177] Trial 5 finished with values: [0.67

29
max_acc: 0.7235788048389716, max_dpd: 0.06026591322496896, min_acc: 0.49999540487634525, min_dpd: 0.0
max dist: 0.01714452065862096, acc_dist: 0.0171321682480249, dpd_dist: 1.2352410596059355e-05
max dist: 0.020423787316457692, acc_dist: 0.010472676638297167, dpd_dist: 0.009951110678160523
max dist: 0.03931663507543255, acc_dist: 0.03923368459217079, dpd_dist: 8.2950483261762e-05
max dist: 0.062166473558842875, acc_dist: 0.0621664734170249, dpd_dist: 1.4181797732120154e-10
max dist: 0.07224387643863875, acc_dist: 0.04579136738440859, dpd_dist: 0.026452509054230154


[I 2025-05-18 02:44:15,906] Trial 1 finished with values: [0.6592731127061295, 0.015923288926082902] and parameters: {'max_depth': 4, 'eta': 0.24465421022559317, 'subsample': 0.9598084555098281, 'colsample_bytree': 0.9952024121289108}.
[I 2025-05-18 02:44:16,214] Trial 2 finished with values: [0.6735716058119124, 0.030937017293698615] and parameters: {'max_depth': 6, 'eta': 0.2063405310874454, 'subsample': 0.6890213405431125, 'colsample_bytree': 0.7973805260778758}.
[I 2025-05-18 02:44:16,480] Trial 3 finished with values: [0.6707165023144106, 0.03431186549613302] and parameters: {'max_depth': 3, 'eta': 0.2224396337428844, 'subsample': 0.693689783353354, 'colsample_bytree': 0.8657117185031096}.
[I 2025-05-18 02:44:16,778] Trial 4 finished with values: [0.6735716058119124, 0.030937017293698615] and parameters: {'max_depth': 6, 'eta': 0.23969391500135154, 'subsample': 0.730640881512773, 'colsample_bytree': 0.8608083853578643}.
[I 2025-05-18 02:44:17,082] Trial 5 finished with values: [0.

Pareto-optimal solutions:
Params: {'alpha': 0, 'max_depth': 5, 'eta': 0.2413907161719555, 'subsample': 0.9281607519593525, 'colsample_bytree': 0.6488199467320804}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.015625, 'max_depth': 6, 'eta': 0.14902745440245718, 'subsample': 0.6528513298518391, 'colsample_bytree': 0.9093575471457058}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.01953125, 'max_depth': 5, 'eta': 0.26940039672587757, 'subsample': 0.608692211587141, 'colsample_bytree': 0.8984966377191745}, Accuracy: 0.5000, DPD: 0.0000
Params: {'alpha': 0.021484375, 'max_depth': 7, 'eta': 0.1539737064064355, 'subsample': 0.6097284977854751, 'colsample_bytree': 0.7542571158699494}, Accuracy: 0.5293, DPD: 0.0002
Params: {'alpha': 0.02367401123046875, 'max_depth': 7, 'eta': 0.010007326109618367, 'subsample': 0.6211134156349097, 'colsample_bytree': 0.6301343102883658}, Accuracy: 0.5335, DPD: 0.0031
Params: {'alpha': 0.023675918579101562, 'max_depth': 6, 'eta': 0.011006896090615087, 's

In [5]:
fig = px.scatter(
    plot_df,
    x='DPD',
    y='Accuracy',
    color='alpha',
    color_continuous_scale='Viridis',
    size_max=15,
    hover_data={'alpha': True, 'DPD': True, 'Accuracy': True},
    title='Pareto Front: DPD vs. Accuracy'
)
fig.update_traces(marker=dict(line_width=0))
fig.show()

In [9]:
import matplotlib.pyplot as plt
import os 
os.makedirs("plots", exist_ok=True)
dpd_vals = [res['dpd'] for res in results]
acc_vals = [res['accuracy'] for res in results]
alpha_vals = [res['alpha'] for res in results]
plt.figure()
plt.scatter(dpd_vals, acc_vals, c=alpha_vals, cmap="viridis")
plt.colorbar(label="alpha")
plt.xlabel("DPD"); plt.ylabel("Accuracy")
out_path = os.path.join("plots", "Pareto Solutions DPD VS Accuracy.png")
plt.savefig(out_path, dpi=150)
plt.close()

In [10]:
import matplotlib.pyplot as plt
import os

# Extract DPD and accuracy values
dpd_vals = [res['dpd'] for res in results]
acc_vals = [res['accuracy'] for res in results]

# ensure output dir
os.makedirs("plots", exist_ok=True)

# sort by DPD so the line is monotonic
pts = sorted(zip(dpd_vals, acc_vals), key=lambda x: x[0])
dpd_sorted, acc_sorted = zip(*pts)

# plot line + red dots
plt.figure()
plt.plot(dpd_sorted, acc_sorted, '-o', color='red')
plt.xlabel("DPD")
plt.ylabel("Accuracy")
plt.title("Pareto Front: DPD vs Accuracy")

out_path = os.path.join("plots", "Pareto Front  DPD vs Accuracy.png")
plt.savefig(out_path, dpi=150)
plt.close()